In [1]:
from preprocessing import *
import preprocessing as prp
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as data_utils
from usad import *

In [2]:
dataframe = pd.read_csv(r"C:\Users\carla\OneDrive\Documenti\GitHub\Unsupervised_Anomaly_Detection_thesis\Data\train_features.csv")
dataframe.shape

(1749494, 57)

In [3]:
df=dataframe[['building_id','primary_use', 'timestamp', 'meter_reading', 'sea_level_pressure', 'is_holiday','anomaly']]
df

,building_id,primary_use,timestamp,meter_reading,sea_level_pressure,is_holiday,anomaly
0,1,Education,2016-01-01 00:00:00,NaN,967.2,1,0
1,32,Office,2016-01-01 00:00:00,NaN,967.2,1,0
2,41,Office,2016-01-01 00:00:00,NaN,967.2,1,0
3,55,Office,2016-01-01 00:00:00,NaN,967.2,1,0
4,69,Parking,2016-01-01 00:00:00,NaN,967.2,1,0
...,...,...,...,...,...,...,...
1749489,1316,Entertainment/public assembly,2016-12-31 23:00:00,38.844,967.2,0,0
1749490,1318,Entertainment/public assembly,2016-12-31 23:00:00,202.893,967.2,0,0
1749491,1319,Entertainment/public assembly,2016-12-31 23:00:00,NaN,967.2,0,0
1749492,1323,Office,2016-12-31 23:00:00,172.000,967.2,0,0


In [4]:
imputed_df = impute_nulls(df)
imputed_df

,building_id,primary_use,timestamp,meter_reading,sea_level_pressure,is_holiday,anomaly
0,1,Education,2016-01-01 00:00:00,31.477940,967.2,1,0
200,1,Education,2016-01-01 01:00:00,29.582970,1019.4,1,0
398,1,Education,2016-01-01 02:00:00,28.635485,1018.8,1,0
597,1,Education,2016-01-01 03:00:00,28.161742,1018.1,1,0
796,1,Education,2016-01-01 04:00:00,27.924871,1019.0,1,0
...,...,...,...,...,...,...,...
1748693,1353,Religious worship,2016-12-31 19:00:00,2.425000,967.2,0,0
1748893,1353,Religious worship,2016-12-31 20:00:00,2.450000,967.2,0,0
1749093,1353,Religious worship,2016-12-31 21:00:00,2.425000,967.2,0,0
1749293,1353,Religious worship,2016-12-31 22:00:00,2.450000,967.2,0,0


Now that we have imputed the missing values for the column containing the energy consumption measurements, we can procees by adding a couple of features more and further imputing the missing dates for each timeseries in the dataset.

In [5]:
df = add_trigonometric_features(imputed_df)
df

,building_id,primary_use,timestamp,meter_reading,sea_level_pressure,is_holiday,anomaly,weekday,weekday_y,weekday_x
0,1,Education,2016-01-01 00:00:00,31.477940,967.2,1,0,4,-0.433884,-0.900969
200,1,Education,2016-01-01 01:00:00,29.582970,1019.4,1,0,4,-0.433884,-0.900969
398,1,Education,2016-01-01 02:00:00,28.635485,1018.8,1,0,4,-0.433884,-0.900969
597,1,Education,2016-01-01 03:00:00,28.161742,1018.1,1,0,4,-0.433884,-0.900969
796,1,Education,2016-01-01 04:00:00,27.924871,1019.0,1,0,4,-0.433884,-0.900969
...,...,...,...,...,...,...,...,...,...,...
1748693,1353,Religious worship,2016-12-31 19:00:00,2.425000,967.2,0,0,5,-0.974928,-0.222521
1748893,1353,Religious worship,2016-12-31 20:00:00,2.450000,967.2,0,0,5,-0.974928,-0.222521
1749093,1353,Religious worship,2016-12-31 21:00:00,2.425000,967.2,0,0,5,-0.974928,-0.222521
1749293,1353,Religious worship,2016-12-31 22:00:00,2.450000,967.2,0,0,5,-0.974928,-0.222521


In [6]:
dfs_dict = impute_missing_dates(df)

In [7]:
dfs_dict

{1:                      building_id primary_use  meter_reading  \
 2016-01-01 00:00:00            1   Education      31.477940   
 2016-01-01 01:00:00            1   Education      29.582970   
 2016-01-01 02:00:00            1   Education      28.635485   
 2016-01-01 03:00:00            1   Education      28.161742   
 2016-01-01 04:00:00            1   Education      27.924871   
 ...                          ...         ...            ...   
 2016-12-31 19:00:00            1   Education      20.006000   
 2016-12-31 20:00:00            1   Education      15.364000   
 2016-12-31 21:00:00            1   Education      15.685000   
 2016-12-31 22:00:00            1   Education      15.925000   
 2016-12-31 23:00:00            1   Education      14.964000   
 
                      sea_level_pressure  is_holiday  anomaly  weekday  \
 2016-01-01 00:00:00               967.2           1        0        4   
 2016-01-01 01:00:00              1019.4           1        0        4   
 2016

In [8]:
df1 = pd.concat(dfs_dict.values())
df1

,building_id,primary_use,meter_reading,sea_level_pressure,is_holiday,anomaly,weekday,weekday_y,weekday_x
2016-01-01 00:00:00,1,Education,31.477940,967.2,1,0,4,-0.433884,-0.900969
2016-01-01 01:00:00,1,Education,29.582970,1019.4,1,0,4,-0.433884,-0.900969
2016-01-01 02:00:00,1,Education,28.635485,1018.8,1,0,4,-0.433884,-0.900969
2016-01-01 03:00:00,1,Education,28.161742,1018.1,1,0,4,-0.433884,-0.900969
2016-01-01 04:00:00,1,Education,27.924871,1019.0,1,0,4,-0.433884,-0.900969
...,...,...,...,...,...,...,...,...,...
2016-12-31 19:00:00,1353,Religious worship,2.425000,967.2,0,0,5,-0.974928,-0.222521
2016-12-31 20:00:00,1353,Religious worship,2.450000,967.2,0,0,5,-0.974928,-0.222521
2016-12-31 21:00:00,1353,Religious worship,2.425000,967.2,0,0,5,-0.974928,-0.222521
2016-12-31 22:00:00,1353,Religious worship,2.450000,967.2,0,0,5,-0.974928,-0.222521


Let's now obtain the train and validation set. We are going to split the dataset into 2 sets, according to the building id.

In [9]:
dfs_train, dfs_val = train_val_split(df1)
train = pd.concat(dfs_train.values())

In [10]:
val = pd.concat(dfs_val.values())

In [11]:
train

,building_id,primary_use,meter_reading,sea_level_pressure,is_holiday,anomaly,weekday,weekday_y,weekday_x
2016-01-01 00:00:00,1,Education,31.477940,967.2,1,0,4,-0.433884,-0.900969
2016-01-01 01:00:00,1,Education,29.582970,1019.4,1,0,4,-0.433884,-0.900969
2016-01-01 02:00:00,1,Education,28.635485,1018.8,1,0,4,-0.433884,-0.900969
2016-01-01 03:00:00,1,Education,28.161742,1018.1,1,0,4,-0.433884,-0.900969
2016-01-01 04:00:00,1,Education,27.924871,1019.0,1,0,4,-0.433884,-0.900969
...,...,...,...,...,...,...,...,...,...
2016-12-31 19:00:00,1353,Religious worship,2.425000,967.2,0,0,5,-0.974928,-0.222521
2016-12-31 20:00:00,1353,Religious worship,2.450000,967.2,0,0,5,-0.974928,-0.222521
2016-12-31 21:00:00,1353,Religious worship,2.425000,967.2,0,0,5,-0.974928,-0.222521
2016-12-31 22:00:00,1353,Religious worship,2.450000,967.2,0,0,5,-0.974928,-0.222521


In [12]:
val

,building_id,primary_use,meter_reading,sea_level_pressure,is_holiday,anomaly,weekday,weekday_y,weekday_x
2016-01-01 00:00:00,69,Parking,19.232375,967.2,1,0,4,-0.433884,-0.900969
2016-01-01 01:00:00,69,Parking,13.457187,1019.4,1,0,4,-0.433884,-0.900969
2016-01-01 02:00:00,69,Parking,10.569594,1018.8,1,0,4,-0.433884,-0.900969
2016-01-01 03:00:00,69,Parking,9.125797,1018.1,1,0,4,-0.433884,-0.900969
2016-01-01 04:00:00,69,Parking,8.403898,1019.0,1,0,4,-0.433884,-0.900969
...,...,...,...,...,...,...,...,...,...
2016-12-31 19:00:00,1319,Entertainment/public assembly,132.040616,967.2,0,0,5,-0.974928,-0.222521
2016-12-31 20:00:00,1319,Entertainment/public assembly,132.040616,967.2,0,0,5,-0.974928,-0.222521
2016-12-31 21:00:00,1319,Entertainment/public assembly,132.040616,967.2,0,0,5,-0.974928,-0.222521
2016-12-31 22:00:00,1319,Entertainment/public assembly,132.040616,967.2,0,0,5,-0.974928,-0.222521


In [13]:
train_window = 168

In [14]:
X, y = split_sequences(train, train_window)

In [15]:
X, y

(array([[[6.08934018e-01, 0.00000000e+00, 0.00000000e+00, 2.77479066e-01,
          1.00000000e+00],
         [5.49435454e-01, 8.28571429e-01, 0.00000000e+00, 2.77479066e-01,
          1.00000000e+00],
         [5.19686172e-01, 8.19047619e-01, 0.00000000e+00, 2.77479066e-01,
          1.00000000e+00],
         ...,
         [4.89936890e-01, 7.71428571e-01, 5.55111512e-17, 7.22520934e-01,
          0.00000000e+00],
         [4.89936890e-01, 0.00000000e+00, 5.55111512e-17, 7.22520934e-01,
          0.00000000e+00],
         [4.89936890e-01, 7.65079365e-01, 5.55111512e-17, 7.22520934e-01,
          0.00000000e+00]],
 
        [[5.49435454e-01, 8.28571429e-01, 0.00000000e+00, 2.77479066e-01,
          1.00000000e+00],
         [5.19686172e-01, 8.19047619e-01, 0.00000000e+00, 2.77479066e-01,
          1.00000000e+00],
         [5.04811531e-01, 8.07936508e-01, 0.00000000e+00, 2.77479066e-01,
          1.00000000e+00],
         ...,
         [4.89936890e-01, 0.00000000e+00, 5.55111512e-17, 7.

In [16]:
X.shape

(1395792, 168, 5)

In [17]:
y.shape

(1395792,)

In [26]:
BATCH_SIZE =  X.shape[0]
N_EPOCHS = 1
hidden_size = 2

In [19]:
w_size = X.shape[1] * X.shape[2]
z_size = X.shape[1] * hidden_size
w_size, z_size

(840, 336)

In [20]:
import torch.utils.data as data_utils

In [21]:
train_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(torch.from_numpy(X).float().view(([X.shape[0], w_size]))), batch_size = BATCH_SIZE, shuffle = False, num_workers = 0)

In [22]:
X_val, y_val = split_sequences(val, train_window)

In [23]:
X_val.shape, y_val.shape

((327408, 168, 5), (327408,))

In [24]:
val_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(torch.from_numpy(X_val).float().view(([X_val.shape[0],w_size]))) , batch_size=X_val.shape[0], shuffle=False, num_workers=0)

In [25]:
model = UsadModel(w_size, z_size)
model = to_device(model,device)

In [27]:
history = training(N_EPOCHS,model,train_loader,val_loader)

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 4689861120 bytes.

In [28]:
plot_history(history)

NameError: name 'history' is not defined

In [ ]:
torch.save({
            'encoder': model.encoder.state_dict(),
            'decoder1': model.decoder1.state_dict(),
            'decoder2': model.decoder2.state_dict()
            }, "model.pth")